In [1]:
import os
import json

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent
from react_agent.src.agents.triage import Triage

from react_agent.src.util.tools_fabric import ToolsFabric

from react_agent.src.util.llm_proxy import LLM_PROXY

from experiments.util.fact_score.fact_scorer import FactScorer

In [2]:
USE_MCP = False
DEBUG_MODE = False

fact_scorer = FactScorer()
path_to_ressources = "./ressources/"

files = {
    "extensibility": "extensibility_assistance_result.json",
    "malaysia": "malaysia_support_result.json",
    "peppol": "peppol_support_result.json",
}

In [3]:
file_path = os.path.abspath(path_to_ressources + files["extensibility"])


with open(file_path, encoding="utf8") as f:
    json_file = json.load(f)

In [4]:
case = json_file["cases"][1]

In [5]:
triage_agent = Triage()

query_categorization = triage_agent.triage_user_message(
    user_message=case["question"]
)
if DEBUG_MODE:
    print(f"Query categorization: {query_categorization}")

In [6]:
tools = ToolsFabric.get_tools_for_category( use_mcp = USE_MCP, configuration = query_categorization["category"] )

In [7]:
if USE_MCP:
    async with MultiServerMCPClient( tools ) as client:
        agent = ReActAgent(tool_list=client.get_tools())
        await agent.arun_and_print_agent_stream(
            user_message=query_categorization["user_query"]
        )
else:
    agent = ReActAgent(tool_list=tools)
    agent_answer = agent.run_agent_with_input(user_message=query_categorization["user_query"], debug=DEBUG_MODE)

In [8]:
print(agent_answer)

### Final Answer

To enhance the generated XML for a Greece customer in Public Cloud before sending it to the Tax Authorities using ADT, you can follow these steps:

1. **Utilize Business Add-Ins (BAdIs)**:
   - **Determine myDATA VAT Classification**: Customize VAT categories in the XML based on transaction locations (Greece, EU, non-EU).
   - **Determine myDATA Classification Details**: Implement custom logic for Income or Expense Classification in the XML file.
   - **Cloud EDOC Adaptor BADI**: Modify the final invoice XML to fill fields not covered by standard mapping and perform error checking.

2. **Integrate with SAP Application Interface Framework (AIF)**:
   - Use AIF for mapping electronic documents to the XML format required by Greek tax authorities, supporting XML file modifications.

3. **Implement Enhancement Spot ES_EDOCUMENT_CLOUD**:
   - **EDOC_ADAPTOR_CLOUD**: Enhance XML mapping by editing or adding fields, and prevent electronic document creation for specific source

In [9]:
output = await fact_scorer.get_fact_score(facts=case.get("facts"), knowledge_source=agent_answer, debug=DEBUG_MODE)
print("Fact score: ", output)

Fact score:  0.5


In [10]:
LLM_PROXY.print_usage()

Call Count: 12
Token Usage: {'input_tokens': 22954, 'output_tokens': 2009, 'total_tokens': 24963}
